In [ ]:
pip install music21 tensorflow


In [3]:
import numpy as np
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import glob
import pickle


In [ ]:
import glob
import pickle
from music21 import converter, instrument, note, chord

songs_path   = '/Users/monilshah/Documents/02_NWU/09_MSDS_458_DL/99_group_project/LSTMMusic/midi_songs/'

def get_notes():
    notes = []

    # Path to your MIDI files stored in Google Drive
    for file in glob.glob(songs_path + "*.mid"):
        midi = converter.parse(file)
        print(f"Parsing {file}")

        parts = instrument.partitionByInstrument(midi)
        if parts:
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    # Save notes to a pickle file
    with open('notes.pkl', 'wb') as f:
        pickle.dump(notes, f)

    return notes

# Generate the notes.pkl file
get_notes()


In [5]:
def prepare_sequences(notes, n_vocab):
    sequence_length = 100  # Length of each training sequence

    # Map unique notes to integers
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    # Prepare input and output sequences
    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape for LSTM: (number of sequences, sequence length, 1)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # Normalize input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output, num_classes=n_vocab)

    return network_input, network_output


In [30]:
def create_model(network_input, n_vocab):
    model = Sequential()

    # LSTM layer
    model.add(LSTM(64, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    #model.add(Dropout(0.1))
    
    model.add(LSTM(64, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    #model.add(Dropout(0.1))
    
    model.add(LSTM(64, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    #model.add(Dropout(0.1))
    
    model.add(LSTM(64, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    #model.add(Dropout(0.1))
    
    #model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    #model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    #model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    #model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    #model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    #model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))

    # Another LSTM layer
    model.add(LSTM(512, return_sequences=False))
    model.add(Dropout(0.1))

    # Fully connected layer
    model.add(Dense(256))
    model.add(Dropout(0.1))
    
    # Fully connected layer
    model.add(Dense(128))
    model.add(Dropout(0.1))

    # Output layer
    model.add(Dense(n_vocab, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model


In [31]:
# def create_network(input_shape):
#     """Build the LSTM model."""
#     model = Sequential()

#     # Add LSTM layers
#     model.add(LSTM(256, input_shape=(input_shape[1], input_shape[2]), return_sequences=True))
#     model.add(Dropout(0.3))

#     model.add(LSTM(256, return_sequences=False))
#     model.add(Dropout(0.3))

#     # Add Dense layers
#     model.add(Dense(256))
#     model.add(Dropout(0.3))

#     model.add(Dense(128))
#     model.add(Dense(88))  # 88 possible notes

#     # Final activation
#     model.add(Activation('softmax'))

#     # Compile the model
#     model.compile(loss='categorical_crossentropy', optimizer='adam')

#     return model


In [32]:
def train_model(model, network_input, network_output, epochs=200, batch_size=64):
    """Train the model with callbacks."""

    # Define callback functions
    checkpoint = ModelCheckpoint(
         'weights-improvement-{epoch:02d}-{loss:.4f}-bigger.keras',
        monitor='loss',
        verbose=1,
        save_best_only=True,
        mode='min'
    )

    early_stopping = EarlyStopping(
        monitor='loss',
        patience=2,
        verbose=1,
        mode='min',
        restore_best_weights=True
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='loss',
        factor=0.2,
        patience=2,
        min_lr=0.001,
        verbose=1
    )

    # Train the model
    model.fit(
        network_input,
        network_output,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint, early_stopping, reduce_lr]
    )


In [ ]:
# Load notes from the saved file
with open('notes.pkl', 'rb') as f:
    notes = pickle.load(f)

#Get the number of unique notes
n_vocab = len(set(notes))

# Prepare sequences for LSTM
network_input, network_output = prepare_sequences(notes, n_vocab)

# Create the LSTM model
model = create_model(network_input, n_vocab)

# Define callbacks
checkpoint = ModelCheckpoint(
    'weights-improvement-{epoch:02d}-{loss:.4f}.keras',  # Save the best model with the extension .keras
    monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss',
    patience=3,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='loss',
    factor=0.2,
    patience=3,
    min_lr=0.0001,
    verbose=1
)

# Train the model with callbacks
model.fit(
    network_input,
    network_output,
    epochs=50,
    batch_size=64,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)


In [20]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input) - 1)

    # Initial input sequence for the model
    pattern = network_input[start]
    generated_notes = []

    # Generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = pitchnames[index]
        generated_notes.append(result)

        # Update pattern
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return generated_notes


In [22]:
def create_midi(generated_notes):
    output_notes = []

    for pattern in generated_notes:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='midi_output/generated_music_v1.mid')


In [23]:
# Load notes, prepare sequences, and train the model (as shown before)
n_vocab = len(set(notes))
#network_input, network_output = prepare_sequences(notes, n_vocab)
#model = create_model(network_input, n_vocab)
#model.fit(network_input, network_output, epochs=200, batch_size=64)

# Generate music
pitchnames = sorted(set(item for item in notes))
generated_notes = generate_notes(model, network_input, pitchnames, n_vocab)

# Convert generated notes to MIDI file
create_midi(generated_notes)
